This file contains offline LM testing. No performant models have been found so far.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "Intel/neural-chat-7b-v3-1"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

tokenizer = AutoTokenizer.from_pretrained(model_id)
# model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", offload_folder="offload")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
instruction = """Imagine I am a robot equiped with a camera and microphone. I am trying to perform a task, and you should help me by sending me commands. You are only allowed to give me the following commands:

- talk(text): robot says text
- move(location): robot moves to the desired location
- turn(angle): robot turns by a given number of degrees

On each step, I will provide you with the objects in the scene and microphone input as a JSON: {microphone: "", objects:[{object_name: "", distance: 0, angle: 0}]}. You should reply with only one command at a time, and you are only allowed to use the commands provided. The distance is in meters, and the direction angle in degrees with respect to the robot's orientation. Negative angles are to the left and positive angles are to the right. If a command is not valid, I will ignore it and ask you for another command. If there is not relevant information in the scene, use the available commands to explore the environment.

Task: go to the person and ask for their name and favourite drink
Objects in the scene: <person, 2 m, 22>
Command:"""

In [10]:
# with open("sample_data/ordering_drinks.txt") as f:
#   instruction = f.read()

messages = [
    {
        "role": "system",
        "content": instruction,
    },
]
usr_input = ''
beginning = False

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

while True:
  if beginning:
    usr_input = input("User: ")
    if len(messages) == 1:
      messages.append({"role": "user", "content": usr_input})
    messages[1]["content"] = usr_input

  # model_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
  # input_length = model_inputs.shape[1]
  # generated_ids = model.generate(model_inputs, max_new_tokens=20)
  # output = tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0]

  inputs = tokenizer(instruction, return_tensors="pt").to("cuda")
  outputs = model_4bit.generate(**inputs, max_new_tokens=20)
  output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(output)
  beginning = True

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Imagine I am a robot equiped with a camera and microphone. I am trying to perform a task, and you should help me by sending me commands. You are only allowed to give me the following commands:

- talk(text): robot says text
- move(location): robot moves to the desired location
- turn(angle): robot turns by a given number of degrees

On each step, I will provide you with the objects in the scene and microphone input as a JSON: {microphone: "", objects:[{object_name: "", distance: 0, angle: 0}]}. You should reply with only one command at a time, and you are only allowed to use the commands provided. The distance is in meters, and the direction angle in degrees with respect to the robot's orientation. Negative angles are to the left and positive angles are to the right. If a command is not valid, I will ignore it and ask you for another command. If there is not relevant information in the scene, use the available commands to explore the environment.

Task: go to the person and ask for the

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 